In [ ]:
import os
import numpy as np
import chromadb
from typing import List, Tuple
from langchain_classic.schema import Document
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from uuid import uuid4

In [2]:
os.listdir()

['01.ipynb']

In [3]:
DATA_PATH = '../data'

os.makedirs( '../chroma_db', exist_ok= True )
CHROMA_PATH = '../chroma_db'

### Define the Chroma Data Base

In [4]:
chroma_client = chromadb.PersistentClient( path = CHROMA_PATH )

In [5]:
collection = chroma_client.get_or_create_collection( name = 'SkillsDynamix_About_Us' )

### Load Documents

In [6]:
loader = PyPDFDirectoryLoader( DATA_PATH )

In [7]:
raw_docs = loader.load()
raw_docs

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2026-02-25T05:19:52+02:00', 'author': 'KimoStore', 'moddate': '2026-02-25T05:19:52+02:00', 'source': '..\\data\\data.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="🏢 Company Overview \nName: Skills Dynamix Tagline: Professional Training Excellence — Master Your Future \nSlogan: Unlock Your Potential, Transform Your Career Description: Skills Dynamix delivers \nworld-class training in Information Technology, English Language (IELTS/TOEFL), and \nBusiness Management across Egypt and the Middle East. Part of: Integrated Training Programs \nof Egypt Founded: 1996 Website: skillsdynamix.com \n \n📊 Key Stats \nMetric Value \nTraining Hours 600,000+ \nSuccess Rate 95% \nYears of Experience 25+ \nExpert Instructors 500+ \nCertification Programs 50+ \nLearning Support 24/7 \nAccess Global Online \n \n🎯 Vision & Mission \nVision: To be the leading catalyst for skills 

### Split text into chunks

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex= True
)

In [9]:
chunks = text_splitter.split_documents( raw_docs )

'ef477f1edcbb46339cbba239ed53d6fb'

In [20]:
def prepare_data_for_chromadb( chunks: List[ Document ] ) -> Tuple[ List[str], List[ dict ], List[ str ] ] :
    
    documents = []
    meta_datas = []
    ids = []
    
    for index, chunk in enumerate( chunks ) :
        
        documents.append( chunk.page_content )
        meta_datas.append( chunk.metadata )
        ids.append( 'doc_' + uuid4().hex + '_' + str( index + 1 ) )
        
    return documents, meta_datas, ids

In [21]:
documents, metadatas, ids = prepare_data_for_chromadb( chunks )

In [23]:
collection.upsert(
    documents= documents,
    metadatas= metadatas,
    ids= ids
)